In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np
import pickle

# Load Data

In [2]:
def load_vectors(pos_file, neg_file):
    """
    Load positive and negative vectors, combine and shuffle them.
    
    Parameters:
    pos_file: str
        Path to file containing positive vectors.
    neg_file: str
        Path to file containing negative vectors.
    
    Returns:
    X: np.array
        Combined and shuffled vectors.
    y: np.array
        Labels for the vectors.
    """
    # Read files
    pos_vectors = np.loadtxt(pos_file)
    neg_vectors = np.loadtxt(neg_file)
    
    # Create labels
    pos_labels = np.ones(len(pos_vectors))
    print(pos_labels)
    neg_labels = -np.ones(len(neg_vectors))
    print(neg_labels)
    
    # Combine data
    X = np.vstack((pos_vectors, neg_vectors))
    y = np.concatenate((pos_labels, neg_labels))
    
    # Shuffle
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    return X[indices], y[indices]

In [3]:
X, y = load_vectors("data/twitter-datasets/train_pos_full_embedding.txt", "data/twitter-datasets/train_neg_full_embedding.txt")

[1. 1. 1. ... 1. 1. 1.]
[-1. -1. -1. ... -1. -1. -1.]


In [11]:
def train_neural_network(X, y, hidden_layers=(64, 32), max_iter=200):
    """Train MLPClassifier with progress tracking"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layers,
        max_iter=max_iter,
        random_state=42,
        solver='adam',
        verbose=True
    )
    
    # Training with progress bar
    print("Training neural network...")
    with tqdm(total=max_iter) as pbar:
        def update_progress(iter_num, loss):
            pbar.update(1)
            pbar.set_description(f'Loss: {loss:.4f}')
        
        # Set custom callback
        model._callback = update_progress
        model.fit(X_train, y_train)
    
    # Evaluation
    y_pred = model.predict(X_test)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return model

In [5]:
def predict_sentiment_nn(model, tweet_vector):
    """Predict sentiment for a single tweet vector"""
    return model.predict([tweet_vector])[0]

# Train model

In [12]:
model = train_neural_network(X, y)

Training neural network...


  0%|          | 0/200 [00:00<?, ?it/s]

Iteration 1, loss = 0.64642539
Iteration 2, loss = 0.63573893
Iteration 3, loss = 0.63137808
Iteration 4, loss = 0.62821997
Iteration 5, loss = 0.62593251
Iteration 6, loss = 0.62420099
Iteration 7, loss = 0.62294746
Iteration 8, loss = 0.62166313
Iteration 9, loss = 0.62065148
Iteration 10, loss = 0.61972772
Iteration 11, loss = 0.61908306
Iteration 12, loss = 0.61824276
Iteration 13, loss = 0.61768044
Iteration 14, loss = 0.61712394
Iteration 15, loss = 0.61653890
Iteration 16, loss = 0.61603562
Iteration 17, loss = 0.61553594
Iteration 18, loss = 0.61519534
Iteration 19, loss = 0.61484738
Iteration 20, loss = 0.61452720
Iteration 21, loss = 0.61413354
Iteration 22, loss = 0.61388404
Iteration 23, loss = 0.61364018
Iteration 24, loss = 0.61329764
Iteration 25, loss = 0.61313312
Iteration 26, loss = 0.61284476
Iteration 27, loss = 0.61264726
Iteration 28, loss = 0.61244147
Iteration 29, loss = 0.61230406
Iteration 30, loss = 0.61210717
Iteration 31, loss = 0.61184224
Iteration 32, los

  0%|          | 0/200 [08:42<?, ?it/s]

Iteration 91, loss = 0.60709115
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.



Classification Report:
              precision    recall  f1-score   support

        -1.0       0.65      0.63      0.64    250091
         1.0       0.64      0.66      0.65    249909

    accuracy                           0.65    500000
   macro avg       0.65      0.65      0.65    500000
weighted avg       0.65      0.65      0.65    500000



In [38]:
# Save model
with open("average_NN_model.pkl", "wb") as f:
    pickle.dump(model, f)

AttributeError: Can't pickle local object 'train_neural_network.<locals>.update_progress'